In [1]:
import probtorch
import torch

import combinators
import hmm
import smc
import utils

In [2]:
generative = smc.ParticleTrace(1)

In [3]:
pi = torch.zeros(1, 5, 5)
for i in range(5):
    n = i + 1
    if n >= 5:
        n = 0
    pi[:, i, n] = 1
mu = torch.arange(5).unsqueeze(0)
sigma = torch.ones(1, 5) / 2
zs = torch.ones(1, 50+1, dtype=torch.long) * -1
zs[:, 0] = 0

In [4]:
for t in range(zs.shape[1] - 1):
    zs, pi, mu, sigma, generative = hmm.hmm_step(zs, pi, mu, sigma, t+1, generative)

In [5]:
generative

Trace{'Z_1': Categorical([torch.LongTensor of size 1]), 'X_1': Normal([torch.FloatTensor of size 1]), 'Z_2': Categorical([torch.LongTensor of size 1]), 'X_2': Normal([torch.FloatTensor of size 1]), 'Z_3': Categorical([torch.LongTensor of size 1]), 'X_3': Normal([torch.FloatTensor of size 1]), 'Z_4': Categorical([torch.LongTensor of size 1]), 'X_4': Normal([torch.FloatTensor of size 1]), 'Z_5': Categorical([torch.LongTensor of size 1]), 'X_5': Normal([torch.FloatTensor of size 1]), 'Z_6': Categorical([torch.LongTensor of size 1]), 'X_6': Normal([torch.FloatTensor of size 1]), 'Z_7': Categorical([torch.LongTensor of size 1]), 'X_7': Normal([torch.FloatTensor of size 1]), 'Z_8': Categorical([torch.LongTensor of size 1]), 'X_8': Normal([torch.FloatTensor of size 1]), 'Z_9': Categorical([torch.LongTensor of size 1]), 'X_9': Normal([torch.FloatTensor of size 1]), 'Z_10': Categorical([torch.LongTensor of size 1]), 'X_10': Normal([torch.FloatTensor of size 1]), 'Z_11': Categorical([torch.LongT

In [6]:
num_particles = 5000

In [7]:
smc_step = smc.smc(hmm.hmm_step, hmm.hmm_retrace)

In [8]:
hmm_params = {
    '\\Pi_0': {
        'concentration': torch.ones(5),
    },
    '\\Pi_1': {
        'concentration': torch.ones(5),
    },
    '\\Pi_2': {
        'concentration': torch.ones(5),
    },
    '\\Pi_3': {
        'concentration': torch.ones(5),
    },
    '\\Pi_4': {
        'concentration': torch.ones(5),
    },
    '\\Pi_5': {
        'concentration': torch.ones(5),
    },
    '\\mu': {
        'loc': mu[0],
        'scale': torch.sqrt(torch.rand(5)),
    },
    '\\sigma': {
        'loc': sigma[0],
        'scale': torch.sqrt(torch.rand(5)),
    }
}

In [ ]:
inference, init_hmm = smc.variational_smc(num_particles, hmm.init_hmm, smc_step, 100, 50, hmm_params, generative, 5)

In [ ]:
inference

In [ ]:
generative

In [ ]:
for t in range(1, zs.shape[1]):
    key = 'Z_%d' % t
    accuracy = (inference[key].value == generative[key].value).sum().item()
    print('SMC accuracy at time %d: %f' % (t, accuracy / num_particles))